# Stage 2a: Generate Alpha Factors using Stock Universe

- Compute custom apha factors
- Compute univeral quant factors
- Compute date information
- Save Alpha Factors

In [1]:
from platform import python_version
import time
from datetime import datetime
import os
import pandas as pd
import numpy as np
import math
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

%matplotlib inline
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (20, 8)

# Set the import path for the tools directiory
import sys
# insert at position 1 in the path, as 0 is the path of this file.
sys.path.insert(1, '../tools')
import importlib
import ameritrade_functions as amc
importlib.reload(amc)
import utils
importlib.reload(utils)

print(f'Python version: {python_version()}')
print(f'Pandas version: {pd.__version__}')

Python version: 3.8.10
Pandas version: 0.25.3


## Configure Ameritrade Information

Ameritrade credentials are stored in environment variables to keep from having unencrypted passwords stored on disk.

The module automatically masks the account numbers to protect the actual accounts. An Ameritrade user can have many investment accounts. We will be working with only one for this demonstration.

In [2]:
username = os.getenv('maiotradeuser')
password = os.getenv('maiotradepw')
client_id = os.getenv('maiotradeclientid')

masked_account_number = '#---9216'
account_portfolios_file_name = 'data/portfolio_data.csv'
portfolio_file_name = 'data/portfolio_' + masked_account_number[-4:] + '.csv'
price_histories_file_name = 'data/price_histories.csv'

# Price History data from Stage 1

In [3]:
price_histories = utils.read_price_histories(price_histories_file_name)
print(f'Date range for price histories: {price_histories.date.min().date()} to {price_histories.date.max().date()}')
close = utils.get_close_values(price_histories)
print(f'You have {len(close.columns)} stocks')
print(close.columns.to_list())
close.tail()

Date range for price histories: 2016-10-04 to 2021-10-04
You have 437 stocks
['A', 'AAL', 'AAP', 'AAPL', 'ABBV', 'ABC', 'ABT', 'ACN', 'ADBE', 'ADI', 'ADM', 'ADP', 'ADSK', 'AEE', 'AES', 'AFL', 'AIG', 'AJG', 'AKAM', 'ALB', 'ALGN', 'ALK', 'ALL', 'ALLE', 'AMAT', 'AMCR', 'AMD', 'AMGN', 'AMP', 'AMT', 'AMZN', 'ANET', 'ANSS', 'ANTM', 'AON', 'APA', 'APD', 'APH', 'ARE', 'ATO', 'ATVI', 'AVB', 'AVGO', 'AVY', 'AWK', 'AXP', 'AZO', 'BA', 'BAC', 'BAX', 'BBY', 'BDX', 'BEN', 'BIIB', 'BIO', 'BK', 'BKNG', 'BKR', 'BLL', 'BMY', 'BR', 'BRO', 'BSX', 'BWA', 'C', 'CAH', 'CARR', 'CAT', 'CB', 'CBRE', 'CCL', 'CDAY', 'CDNS', 'CDW', 'CE', 'CERN', 'CF', 'CFG', 'CHD', 'CHTR', 'CI', 'CINF', 'CL', 'CLX', 'CMA', 'CMCSA', 'CME', 'CMG', 'CMI', 'CMS', 'CNC', 'CNP', 'COF', 'COO', 'COP', 'COST', 'CPB', 'CPRT', 'CRL', 'CRM', 'CSCO', 'CSX', 'CTAS', 'CTLT', 'CTSH', 'CTVA', 'CTXS', 'CVS', 'CVX', 'CZR', 'D', 'DAL', 'DD', 'DE', 'DFS', 'DG', 'DHI', 'DHR', 'DIS', 'DISCA', 'DISCK', 'DISH', 'DLR', 'DLTR', 'DOV', 'DOW', 'DRE', 'DRI', 'D

ticker,A,AAL,AAP,AAPL,ABBV,ABC,ABT,ACN,ADBE,ADI,...,WYNN,XEL,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
date,,,,,,,,,,,,,,,,,,,,,
2021-09-28 00:00:00+00:00,159.900,21.43,214.78,141.91,107.340,121.290,118.74,324.21,578.7700,171.29,...,84.3100,61.42,59.92,59.14,128.920,122.54,150.85,531.18,62.74,192.77
2021-09-29 00:00:00+00:00,160.000,21.01,216.77,142.83,108.840,122.500,119.46,323.53,577.7000,168.84,...,83.6300,62.67,59.88,58.43,127.780,123.04,149.36,525.67,63.74,194.99
2021-09-30 00:00:00+00:00,157.530,20.52,208.89,141.50,107.870,119.450,118.13,319.92,575.7200,167.48,...,84.7500,62.50,58.82,58.05,123.680,122.31,146.36,515.42,61.89,194.14
2021-10-01 00:00:00+00:00,155.990,21.65,208.00,142.65,109.090,119.810,117.21,324.86,577.4700,167.37,...,87.2100,62.49,60.93,59.07,121.300,123.64,149.47,510.98,63.65,196.19
2021-10-04 00:00:00+00:00,155.405,21.78,208.05,140.95,109.601,120.185,117.14,322.99,571.0875,166.94,...,86.9487,62.90,61.72,59.41,121.445,124.06,149.35,508.08,64.04,193.98


# Factors

### General
- Momentum
- Mean Reversion
- Overnight Sentiment

### Universal Quant Features
- Volatility 
- Dollar Volume
- Market Dispersion
- Market Volatility
- Date Parts

#TODO: SMA for market disp/vol

In [4]:
import trading_factors as alpha_factors
importlib.reload(alpha_factors)

all_factors = pd.concat(
[
    alpha_factors.OvernightSentiment(price_histories, 5).demean().rank().zscore().smoothed(10).rank().zscore().for_al(),
    alpha_factors.FactorMeanReversion(price_histories, 5).demean().rank().zscore().for_al(),
    alpha_factors.AnnualizedVolatility(price_histories, 20).rank().zscore().for_al()
], axis=1)
all_factors.sort_index(inplace=True)
all_factors = all_factors.dropna()

all_factors.to_csv('data/all_factors.csv')

In [5]:
all_factors = pd.read_csv('data/all_factors.csv', parse_dates=['date'])
all_factors.set_index(['date', 'ticker'], inplace=True)
all_factors.sort_index(inplace=True)
all_factors.iloc[all_factors.index.get_level_values('ticker') == 'AAL'].tail()

,,overnight_sentiment_5_day_smoothed,mean_reversion_5_day_logret,annualzed_volatility_20_day
date,ticker,,,
2021-09-28 00:00:00+00:00,AAL,1.664676,-1.403084,1.411011
2021-09-29 00:00:00+00:00,AAL,1.672603,-0.760995,1.426865
2021-09-30 00:00:00+00:00,AAL,1.688457,-0.388424,1.426865
2021-10-01 00:00:00+00:00,AAL,1.680530,-0.990878,1.387230
2021-10-04 00:00:00+00:00,AAL,1.672603,-1.260397,1.347595


In [6]:
all_factors.describe()

,overnight_sentiment_5_day_smoothed,mean_reversion_5_day_logret,annualzed_volatility_20_day
count,5.331400e+05,5.331400e+05,5.331400e+05
mean,-1.599301e-19,-1.599301e-19,-3.831658e-20
std,1.000001e+00,1.000001e+00,1.000001e+00
min,-1.728092e+00,-1.728120e+00,-1.728092e+00
25%,-8.640459e-01,-8.640485e-01,-8.640485e-01
50%,0.000000e+00,0.000000e+00,0.000000e+00
75%,8.640459e-01,8.640485e-01,8.640485e-01
max,1.728092e+00,1.728120e+00,1.728102e+00


In [7]:
all_factors.head()

overnight_sentiment_5_day_smoothed  \
date                      ticker                                       
2016-11-29 00:00:00+00:00 A                                 0.483549   
                          AAL                               1.078076   
                          AAP                               1.593332   
                          AAPL                             -1.482354   
                          ABBV                             -0.396351   

                                  mean_reversion_5_day_logret  \
date                      ticker                                
2016-11-29 00:00:00+00:00 A                          1.030522   
                          AAL                       -0.768928   
                          AAP                       -0.761001   
                          AAPL                       0.578678   
                          ABBV                      -0.840271   

                                  annualzed_volatility_20_day  
date                      ticker                               
2016-11-29 00:00:00+00:00 A                         -0.277448  
                          AAL                       -0.126833  
                          AAP                        1.664685  
                          AAPL                      -1.078082  
                          ABBV                       0.864051

In [8]:
all_factors.tail()

overnight_sentiment_5_day_smoothed  \
date                      ticker                                       
2021-10-04 00:00:00+00:00 YUM                               0.206103   
                          ZBH                              -0.142686   
                          ZBRA                             -0.895754   
                          ZION                              1.514062   
                          ZTS                              -0.404278   

                                  mean_reversion_5_day_logret  \
date                      ticker                                
2021-10-04 00:00:00+00:00 YUM                       -0.229884   
                          ZBH                       -0.103051   
                          ZBRA                       1.617113   
                          ZION                      -1.292105   
                          ZTS                        0.126832   

                                  annualzed_volatility_20_day  
date                      ticker                               
2021-10-04 00:00:00+00:00 YUM                       -1.537843  
                          ZBH                       -1.411011  
                          ZBRA                       1.442719  
                          ZION                      -0.126832  
                          ZTS                        0.364643